# Exploratory data analysis

In [6]:
# Import necessary libraries
import pandas as pd
import numpy as np
from colorama import Fore, Style
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import sys
import os

ModuleNotFoundError: No module named 'colorama'

## 1. Loading the Data
- Importing the preprocessed data


In [7]:
#(from ml_logic.preprocessor import preprocess_features)

# Import the preprocessing function
sys.path.append('..')  # Adjust the path to your file structure
from demand_predictor.ml_logic.preprocessor import preprocess_features  # Adjust the import according to your file structure

# Define the path to your raw data CSV file
data_path = '../raw_data/hotel_bookings_raw.csv'  # Adjust the path if necessary

# Load the data
df = pd.read_csv(data_path)

# Display the first few rows of the dataframe
print(df.head())

# Preprocess the data
X_processed = preprocess_features(df)

# Display the shape of the processed data
print("Processed data shape:", X_processed.shape)

          hotel  is_canceled  lead_time  arrival_date_year arrival_date_month  \
0  Resort Hotel            0        342               2015               July   
1  Resort Hotel            0        737               2015               July   
2  Resort Hotel            0          7               2015               July   
3  Resort Hotel            0         13               2015               July   
4  Resort Hotel            0         14               2015               July   

   arrival_date_week_number  arrival_date_day_of_month  \
0                        27                          1   
1                        27                          1   
2                        27                          1   
3                        27                          1   
4                        27                          1   

   stays_in_weekend_nights  stays_in_week_nights  adults  ...  INFLATION  \
0                        0                     0       2  ...        1.8   
1           

## 2. Data Visualization
- Univariate analysis
- Bivariate analysis (scatterplot)
- Multivariate analysis
- Autogluon summary
- Histograms and Density PLots
- Boxplots
- Heatmaps
- Bar Plots
- Pair Plots
- Correlation Matrix
- Pairwise Correlation
- etc.


In [ ]:
# #Summarize the dataset using AutoGluon
# from autogluon.tabular import TabularDataset, TabularPredictor
# ag_data = TabularDataset(data_path)
# print(ag_data.describe())

## 3. Feature Engineering
- Creating new features
- Transforming existing features

## 4. Summary and Next Steps
- Key insights from the analysis
- Potential next steps for modeling